In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM
import torch
import time

BASE_MODEL = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"
MODEL_PATH = "../models/checkpoint-3"

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

# Загружаем модель с адаптацией LoRA
model = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True  # обязательно!
)

model.eval()

# Определим генератор
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)

# Формируем prompt в формате Qwen
SYSTEM_PROMPT = "Ты — преподаватель немецкого языка для русскоязычных студентов. Объясняй грамотно, понятно, без лишней воды."

def build_prompt(user_input):
    return (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n{user_input}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

# Функция генерации

def generate_response(prompt: str, max_tokens=256):
    with torch.no_grad():
        out = generator(prompt, max_new_tokens=max_tokens)[0]["generated_text"]
    return out[len(prompt):].strip()

# Пример вызова
prompt = build_prompt("Объясни разницу между wissen и kennen на A2 уровне")

start = time.time()
response = generate_response(prompt, max_tokens=1024)
print("⏱️ Время генерации:", round(time.time() - start, 2), "сек")
print("\U0001F4AC", response)


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1161: character maps to <undefined>